## Step 1: Import necessary modules and load configurations

In [1]:
import sys
import json
import os
import torch
from transformers import (
    AutoTokenizer, 
    AutoModelForCausalLM, 
    Trainer, 
    TrainingArguments, 
    DataCollatorForSeq2Seq, 
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model
from datasets import Dataset

os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"
# Load configuration settings
from config import TOKENIZER_PATH, MODEL_PATH, PIPELINE_PARAMS, QLORA_PARAMS, ALPACA_LORA_PROMPTS_USER_PROFILE_AND_CANDIDATE_ITEMS
from utils import *

# Verification
print("Configuration Loaded:")
print("Tokenizer Path:", TOKENIZER_PATH)
print("Model Path:", MODEL_PATH)
print("Pipeline Parameters:", PIPELINE_PARAMS)
print("QLoRA Parameters:", QLORA_PARAMS)
print("Prompt Template:", ALPACA_LORA_PROMPTS_USER_PROFILE_AND_CANDIDATE_ITEMS)


c:\Users\Trung\anaconda3\envs\torch_recommender\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Configuration Loaded:
Tokenizer Path: models/hf-frompretrained-download/meta-llama/Meta-Llama-3-8B-Instruct
Model Path: models/hf-frompretrained-downloadmeta-llama/Meta-Llama-3-8B-Instruct
Pipeline Parameters: {'max_length': 2048, 'num_return_sequences': 1, 'temperature': 0.7, 'top_k': 50, 'top_p': 0.95, 'repetition_penalty': 1.2}
QLoRA Parameters: {'lora_r': 8, 'lora_alpha': 16, 'lora_dropout': 0.05, 'lora_target_modules': ['q_proj', 'v_proj'], 'gradient_accumulation_steps': 2, 'lora_num_epochs': 2, 'lora_val_iterations': 100, 'lora_early_stopping_patience': 10, 'lora_lr': 0.0001, 'lora_micro_batch_size': 1}
Prompt Template: {'instruction': "### Instruction:\n You are a recommender system specialized in creating user profiles. Your task is to create a comprehensive user profile and a list of candidate items based on a user's review. The user Profile consists of: 1. Short-term Interests: Examine the user's most recent items along with their personalized descriptions 2. Long-term Prefer

## Step 2: Load and verify training data

In [2]:
data_path = "QLoRa_finetuning/matching_ids_chatGPT.json"

# Load the training data
with open(data_path, "r") as file:
    training_data = json.load(file)

# Sample a couple of data points to verify format
print("Training Data Sample:", training_data[:2])  # Display first two entries
# Clear cache before loading model
#torch.cuda.empty_cache()
# Verify data structure
print("Data Structure Verification:")
for i, sample in enumerate(training_data[:2]):
    assert "User_ID" in sample, f"User_ID missing in sample {i}"
    assert "User_Profile" in sample, f"User_Profile missing in sample {i}"
    assert "Candidate_Items" in sample, f"Candidate_Items missing in sample {i}"
print("Data verification successful!")

Training Data Sample: [{'User_ID': 'AFQQQ5LGNSQUEBGDCYBAZZE5T3DA', 'User_Profile': '"Short-Term Interests": The user recently reviewed products focused on facial skincare, specifically creams and serums targeting anti-aging concerns such as fine lines, wrinkles, and dehydration. They showed interest in natural and organic ingredients, vegan-friendly options, and cruelty-free practices. Their preference leans towards face washes and moisturizers that cater to specific skin concerns like acne-prone, sensitive, and dry skin.\n"Long-Term Preferences": Based on the entirety of their reviews, some common denominators emerge:\n* Focus on natural ingredients, particularly plant-based extracts, essential oils, and herbal remedies\n* Concern for anti-aging and rejuvenation, seeking effective solutions for maintaining healthy-looking skin\n* Interest in sheet masks, especially those with unique features like eye patches and targeted application areas\n* Appreciation for moisturized and hydrated s

Training Data Sample: [{'User_ID': 'AFQQQ5LGNSQUEBGDCYBAZZE5T3DA', 'User_Profile': '"Short-Term Interests": The user recently reviewed products focused on facial skincare, specifically creams and serums targeting anti-aging concerns such as fine lines, wrinkles, and dehydration. They showed interest in natural and organic ingredients, vegan-friendly options, and cruelty-free practices. Their preference leans towards face washes and moisturizers that cater to specific skin concerns like acne-prone, sensitive, and dry skin.\n"Long-Term Preferences": Based on the entirety of their reviews, some common denominators emerge:\n* Focus on natural ingredients, particularly plant-based extracts, essential oils, and herbal remedies\n* Concern for anti-aging and rejuvenation, seeking effective solutions for maintaining healthy-looking skin\n* Interest in sheet masks, especially those with unique features like eye patches and targeted application areas\n* Appreciation for moisturized and hydrated skin, often mentioning specific requirements like deep hydration and non-sticky textures\n* Willingness to explore various brands and products, demonstrating adaptability and open-mindedness\n"User_Profile": Our user appears to be someone who prioritizes natural and organic approaches to skincare while focusing on addressing specific skin concerns. They exhibit a willingness to experiment with diverse products and techniques to achieve optimal results. As they continue exploring the world of skincare, they may gravitate toward more niche markets, such as bespoke formulas tailored to individual skin types or advanced technology-driven treatments. For now, our recommendation would focus on recommending products featuring natural, sustainable, and innovative formulations catering to their varied skin needs, including hydration, anti-aging, and sensitivity management. A suggested next step could involve introducing them to emerging trends in customized skincare regimens and cutting-edge technologies for enhanced customer experience.', 'Candidate_Items': {'1': 'Nourishing Skin Serenity', '2': 'Botanical Beauty Discovery', '3': 'Hydrating Harmony Regimen', '4': 'Natural Radiance Revival', '5': 'Advanced Anti-Aging Adaptations'}}, {'User_ID': 'AFSHXT5PTGDSFW2725SDXIE6ZVEA', 'User_Profile': '"Short-Term Interests": Based on the latest review, we see that the user recently engaged with skincare products, specifically vitamin C foam cleansers. Their experience suggests that they value effectiveness in cleansing without strong scents, possibly indicating a preference for subtle fragrances or unscented products. Additionally, their interest in facial care might imply a concern for skin health and wellness.\n"Long-Term Preferences": Our analysis reveals that this user tends towards trying new products with unique characteristics. For instance:\n* In their early engagement with AG Care Natural Remedy Apple Cider Vinegar Leave-On Mist, they appreciated its distinct, fruity fragrance and connection to nature-based ingredients.\n* When exploring perfumes through PINROSE Perfumes Bold Soul, they demonstrated curiosity about unusual aromas and willingness to experiment with different scents.\n* While reviewing microfiber hair towels, their focus lay on functionality, comfort, and ease of use, suggesting practicality plays a significant role in their purchasing decisions.\nFrom this historical context, we can infer that our user is drawn to unconventional products, enjoys discovering novel sensations, and values functional design. They likely prioritize quality materials, innovative features, and gentle, non-overpowering fragrances.\n"User_Profile": This user exhibits a curious and adventurous approach to consumerism, often seeking experiences beyond mainstream choices. With a growing emphasis on self-care and personal grooming, they gravitate toward products offering effective solutions and attention-grabbing attributes. Our analysis indicates that they tend to appreciate subtlety in scents, preferring understated or unscented options. As they continue navigating diverse product offerings, we predict that their enthusiasm for exploration and discovery will remain a defining characteristic, driving them towards the next exciting find.', 'Candidate_Items': {'1': 'Experiential Wellness Treats', '2': 'Niche Fragrance Discoveries', '3': 'Functional Grooming Tools', '4': 'Unconventional Home Remedies', '5': 'Artisanal Skin & Body Care Sets'}}]
Data Structure Verification:
Data verification successful!


## Step 3: Initialize the Tokenizer and Model with Quantization

In [3]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)
tokenizer.pad_token = tokenizer.eos_token  # Set padding token to EOS if not already set

# Set 4-bit quantization configuration for memory efficiency
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',  # Use NormalFloat4 for better memory efficiency
    bnb_4bit_use_double_quant=True  # Double quantization for more memory saving
)

# Load model with quantization
model = AutoModelForCausalLM.from_pretrained(
    MODEL_PATH,
    quantization_config=bnb_config,
    device_map="auto"  # Automatically maps layers to available GPU memory
)



Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
c:\Users\Trung\anaconda3\envs\torch_recommender\lib\site-packages\transformers\quantizers\auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Loading checkpoint shards: 100%|██████████| 2/2 [00:02<00:00,  1.12s/it]


## Step 4: Preprocessing Function to Match Reviews with Profiles

In [4]:
# Load user reviews
reviews_path = "new_data/new_train_output.json"
with open(reviews_path, "r") as file:
    reviews_data = json.load(file)

# Index reviews by user_id for easy matching
reviews_by_user = {entry["user_id"]: entry["reviews"] for entry in reviews_data}

# Preprocess function to format the data for fine-tuning
def preprocess_function(profile_sample):
    user_id = profile_sample["User_ID"]
    
    # Retrieve and format user reviews as input
    reviews = reviews_by_user.get(user_id, [])
    sorted_reviews = sorted(reviews, key=lambda x: x['timestamp'])
# Get the last 10 reviews
    latest_reviews = sorted_reviews[-10:]
    # Format the latest reviews
    review_texts = [f"Product: {review['product_name']}\nRating: {review['rating']}\nTitle: {review['title']}\nReview: {review['text']}\n" 
                    for review in latest_reviews]

    review_texts = review_texts[-10:]
    formatted_reviews = "\n".join(review_texts)
    
    # Instruction prompt
    instruction = ALPACA_LORA_PROMPTS_USER_PROFILE_AND_CANDIDATE_ITEMS['instruction']
    input_text = f"User reviews:\n{formatted_reviews}" if formatted_reviews else "No reviews available for this user."
    output_alpaca = ALPACA_LORA_PROMPTS_USER_PROFILE_AND_CANDIDATE_ITEMS['output']
    # Combine user profile and candidate items as the response
    user_profile_text = profile_sample["User_Profile"]
    candidate_items_text = "\n".join(
        [f"{i + 1}. {item}" for i, item in enumerate(profile_sample["Candidate_Items"].values())]
    )
    
    output_text = f"User Profile:\n{user_profile_text} \nCandidate Items:\n{candidate_items_text} "
    
    # Complete prompt
    full_text = f"\n{instruction}\n\n{input_text}\n\n\n{output_alpaca}\n{output_text}"
    return full_text

print("Preprocessed Sample:", preprocess_function(training_data[0]))

Preprocessed Sample: 
### Instruction:
 You are a recommender system specialized in creating user profiles. Your task is to create a comprehensive user profile and a list of candidate items based on a user's review. The user Profile consists of: 1. Short-term Interests: Examine the user's most recent items along with their personalized descriptions 2. Long-term Preferences: Analyze all reviews and items from the user’s entire history to capture deeper, stable interests that define the user’s lasting preferences. Use this comprehensive historical data to outline consistent themes and inclinations that have remained steady over time. 3. User Profile Summary: Synthesize these findings into a concise profile (max 200 words) that combines insights from both short-term interests and long-term preferences. Use the short-term interests as a query to refine or highlight relevant themes from the long-term history, and provide a cohesive picture of the user’s tastes, typical habits, and potential

## Step 5: Tokenize and Prepare Data

In [5]:
def tokenize_function(sample):
    processed_text = preprocess_function(sample)
    tokenized = tokenizer(
        processed_text,
        truncation=True,
        max_length=PIPELINE_PARAMS['max_length'],
        padding="max_length",
        return_tensors="pt"
    )
    tokenized["labels"] = tokenized["input_ids"].clone()  # Set labels identical to input_ids
    return tokenized

# Set up two different training sizes: 16 and 32 samples

lora_config = LoraConfig(
    r=QLORA_PARAMS['lora_r'],
    lora_alpha=QLORA_PARAMS['lora_alpha'],
    lora_dropout=QLORA_PARAMS['lora_dropout'],
    target_modules=QLORA_PARAMS['lora_target_modules'],
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
batch_size = 16




## Step 6: Configure LoRA and Training Parameters

In [6]:
# Set up two different training sizes: 16 and 32 samples
training_sizes = [16,32,64]

for train_size in training_sizes:
    # Split the dataset
    train_data = training_data[:train_size]
    eval_data = training_data[train_size:train_size + int(0.2 * train_size)]  # 20% of training data as eval

    # Tokenize datasets
    train_tokenized_data = [tokenize_function(sample) for sample in train_data]
    eval_tokenized_data = [tokenize_function(sample) for sample in eval_data]

    # Convert to Dataset format
    train_dataset = Dataset.from_dict({
        "input_ids": [x["input_ids"][0] for x in train_tokenized_data],
        "attention_mask": [x["attention_mask"][0] for x in train_tokenized_data],
        "labels": [x["labels"][0] for x in train_tokenized_data]
    })
    eval_dataset = Dataset.from_dict({
        "input_ids": [x["input_ids"][0] for x in eval_tokenized_data],
        "attention_mask": [x["attention_mask"][0] for x in eval_tokenized_data],
        "labels": [x["labels"][0] for x in eval_tokenized_data]
    })

    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"outputs/best_model_up_ci_{train_size}_samples",  # Distinct directory for each model
        per_device_train_batch_size=1,
        gradient_accumulation_steps=QLORA_PARAMS['gradient_accumulation_steps'],
        num_train_epochs=QLORA_PARAMS['lora_num_epochs'],
        evaluation_strategy="steps",
        eval_steps=QLORA_PARAMS['lora_val_iterations'],
        save_steps=QLORA_PARAMS['lora_val_iterations'],
        logging_steps=10,
        learning_rate=QLORA_PARAMS['lora_lr'],
        save_total_limit=2,
        load_best_model_at_end=False,
        dataloader_pin_memory=False,
        report_to="none",
        fp16=True  # Enable FP16 mixed precision
    )

    # Data collator
    data_collator = DataCollatorForSeq2Seq(tokenizer, pad_to_multiple_of=8, padding=True)

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=eval_dataset,
        data_collator=data_collator
    )

    # Clear GPU cache before training
    torch.cuda.empty_cache()

    # Start training
    print(f"Starting training with {train_size} samples.")
    trainer.train()

    # Save the model and tokenizer in separate directories for each training size
    model.save_pretrained(f"outputs/best_model_up_ci_{train_size}_samples")
    tokenizer.save_pretrained(f"outputs/best_model_up_ci_{train_size}_samples")
    print(f"Model trained with {train_size} samples saved to outputs/best_model_up_ci_{train_size}_samples.")

c:\Users\Trung\anaconda3\envs\torch_recommender\lib\site-packages\transformers\training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Starting training with 16 samples.


 62%|██████▎   | 10/16 [09:01<05:33, 55.66s/it]

{'loss': 4.9944, 'grad_norm': 13.984397888183594, 'learning_rate': 5.6250000000000005e-05, 'epoch': 1.25}


100%|██████████| 16/16 [14:34<00:00, 54.68s/it]


...

{'loss': 1.1895, 'grad_norm': 0.6733031868934631, 'learning_rate': 2.1875e-05, 'epoch': 1.56}


 94%|█████████▍| 60/64 [57:23<04:04, 61.21s/it]

{'loss': 1.2437, 'grad_norm': 0.7231120467185974, 'learning_rate': 6.25e-06, 'epoch': 1.88}


100%|██████████| 64/64 [1:01:27<00:00, 57.61s/it]

{'train_runtime': 3687.1073, 'train_samples_per_second': 0.035, 'train_steps_per_second': 0.017, 'train_loss': 1.3637834638357162, 'epoch': 2.0}
Model trained with 64 samples saved to outputs/best_model_up_ci_64_samples.
